- https://github.com/jiesihu/AUC_Delongtest__python/blob/main/compare_auc_delong.py

In [1]:
from compare_auc_delong import *

In [2]:
labels = ['A-line', 'total-B-line', 'Consolidation', 'Pleural effusion']

# calculate p-value

In [4]:
def cal_pvalue(labels, gt, method1_df, method2_df):
    p_values = {}
    for idx in range(len(labels)):
        method1_outputs = method1_df.iloc[:, idx]
        method2_outputs = method2_df.iloc[:, idx]
        
        gts = gt.iloc[:, idx]
        
        p_value = delong_roc_test(gts, method1_outputs, method2_outputs)
        p_values[labels[idx]] = p_value.item()
        
    return p_values

In [5]:
def cal_CI(gt_df, method_df):
    CI_values = {}
    auc_values = {}
    for idx in range(len(labels)):
        gt_series = gt_df.iloc[:, idx]
        method__series = method_df.iloc[:, idx]
        
        auc, ci = delong_auc_ci(gt_series, method__series)
        
        CI_values[labels[idx]] = ci
        auc_values[labels[idx]] = auc.item()
        
    return CI_values, auc_values

# auc confidence interval 

In [27]:
from scipy.stats import norm

def delong_auc_ci(y_true, y_pred, alpha=0.95):
    """
    DeLong 방법을 이용하여 AUC와 95% 신뢰 구간 계산
    Args:
    - y_true: 실제 레이블 값 (0과 1로 구성된 numpy 배열)
    - y_pred: 예측 확률값 (numpy 배열)
    - alpha: 신뢰수준 (기본값 0.95)

    Returns:
    - auc: AUC 값
    - ci: AUC에 대한 신뢰 구간 (하한, 상한)
    """
    
    y_true = np.array(y_true)  # Convert to NumPy array
    y_pred = np.array(y_pred)  # Convert to NumPy array
    
    auc, auc_var = delong_roc_variance(y_true, y_pred)
    auc_std = np.sqrt(auc_var)

    # 신뢰 구간 계산
    lower_upper_q = np.array([0.5 - alpha / 2, 0.5 + alpha / 2])
    ci = norm.ppf(lower_upper_q, loc=auc, scale=auc_std)
    
    # 신뢰 구간의 범위를 0과 1 사이로 조정
    ci[ci < 0] = 0
    ci[ci > 1] = 1

    return auc, ci

# internal test

## image encoder: densenet 161

In [250]:
fold_num = 3
batch_size = 4

# densenet161_32_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch32_fold_3_densenet161_output.csv')
densenet161_128_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch128_fold_{fold_num}_densenet161_output.csv')

C3D_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch4_fold_{fold_num}_C3D_output.csv')
R2Plus1D_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch4_fold_{fold_num}_R2Plus1D_output.csv')
CNNLSTM_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch4_fold_{fold_num}_CNNLSTM_output.csv')

USVN_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch4_fold_{fold_num}_USVN_output.csv')

LUVM_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch4_fold_{fold_num}_LUVM_v4_____8head_13ksize_output.csv')
models_gt = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_fold_{fold_num}_video_level_gt.csv')

### LUV_Net vs LUV_Net_resnet

In [67]:
# data_type = 'internal_test'
data_type = 'temporally_separated_test'

fold_num = 4

# encoder_resnet50
# /data2/hoon2/Results/model_output_csv/video_level/internal_test/resnet_50/5_artifacts_batch4_fold_0_LUV_Net_8head_13ksize_output.csv
LUV_Net_resnet_outputs = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/resnet_50/5_artifacts_batch4_fold_{fold_num}_LUV_Net_8head_13ksize_output.csv')
LUV_Net_outputs = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/densenet_161/5_artifacts_batch4_fold_{fold_num}_LUV_Net_8head_13ksize_output.csv')

# models_gt = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/densenet_161/5_artifacts_fold_{fold_num}_video_level_gt.csv')
models_gt = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/resnet_50/5_artifacts_fold_{fold_num}_video_level_gt.csv')

In [68]:
CIs, AUCs = cal_CI(models_gt, LUV_Net_resnet_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.832976,0.787994,0.877958
1,total-B-line,0.773305,0.726852,0.819757
2,Consolidation,0.830765,0.783351,0.878179
3,Pleural effusion,0.838263,0.748412,0.928114


In [69]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUV_Net_outputs, LUV_Net_resnet_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,1.784143e-01
1,total-B-line,6.818312e-09
2,Consolidation,3.968055e-01
3,Pleural effusion,1.494125e-01


### LUV_Net vs swin3d_b

In [155]:
# data_type = 'internal_test'
data_type = 'temporally_separated_test'

fold_num = 0

# /data2/hoon2/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch4_fold_0_swin3d_b_output.csv
SwinT_outputs = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/densenet_161/5_artifacts_batch4_fold_{fold_num}_swin3d_b_output.csv')
LUV_Net_outputs = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/densenet_161/5_artifacts_batch4_fold_{fold_num}_LUV_Net_8head_13ksize_output.csv')

# models_gt = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/densenet_161/5_artifacts_fold_{fold_num}_video_level_gt.csv')
models_gt = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/densenet_161/5_artifacts_fold_{fold_num}_video_level_gt.csv')

In [156]:
CIs, AUCs = cal_CI(models_gt, SwinT_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.673838,0.611322,0.736354
1,total-B-line,0.578630,0.521447,0.635813
2,Consolidation,0.697834,0.633860,0.761809
3,Pleural effusion,0.737468,0.639405,0.835530


In [157]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUV_Net_outputs, SwinT_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,1.936759e-05
1,total-B-line,9.103829e-15
2,Consolidation,6.478213e-03
3,Pleural effusion,5.095362e-07


### LUV_Net vs MViT-B

In [137]:
# data_type = 'internal_test'
data_type = 'temporally_separated_test'

fold_num = 4

# /data2/hoon2/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch4_fold_0_swin3d_b_output.csv
mvit_outputs = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/densenet_161/5_artifacts_batch4_fold_{fold_num}_mvit_v1_b_output.csv')
LUV_Net_outputs = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/densenet_161/5_artifacts_batch4_fold_{fold_num}_LUV_Net_8head_13ksize_output.csv')

# models_gt = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/densenet_161/5_artifacts_fold_{fold_num}_video_level_gt.csv')
models_gt = pd.read_csv(f'/data2/hoon2/Results/model_output_csv/video_level/{data_type}/densenet_161/5_artifacts_fold_{fold_num}_video_level_gt.csv')

In [138]:
CIs, AUCs = cal_CI(models_gt, mvit_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.558377,0.482287,0.634468
1,total-B-line,0.492161,0.434389,0.549933
2,Consolidation,0.609815,0.546239,0.673392
3,Pleural effusion,0.820302,0.753208,0.887396


In [139]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUV_Net_outputs, mvit_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,1.852711e-09
1,total-B-line,0.000000e+00
2,Consolidation,1.494942e-10
3,Pleural effusion,4.626061e-02


### LUVM vs USVN

In [252]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, USVN_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.452193
1,total-B-line,0.620615
2,Consolidation,0.005290
3,Pleural effusion,0.000094


In [253]:
CIs, AUCs = cal_CI(models_gt, LUVM_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.919249,0.890538,0.947960
1,total-B-line,0.855372,0.817426,0.893319
2,Consolidation,0.881966,0.848136,0.915795
3,Pleural effusion,0.974974,0.957836,0.992112


In [254]:
CIs, AUCs = cal_CI(models_gt, USVN_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.924419,0.895001,0.953836
1,total-B-line,0.849389,0.812951,0.885828
2,Consolidation,0.851728,0.815190,0.888266
3,Pleural effusion,0.950913,0.929957,0.971870


### LUVM vs C3D

In [255]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, C3D_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,1.283040e-03
1,total-B-line,3.800449e-11
2,Consolidation,1.138399e-07
3,Pleural effusion,1.631554e-05


In [256]:
CIs, AUCs = cal_CI(models_gt, C3D_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.858794,0.821679,0.895910
1,total-B-line,0.670024,0.615200,0.724849
2,Consolidation,0.794646,0.752832,0.836461
3,Pleural effusion,0.929865,0.902317,0.957413


### LUVM vs R2plus1D

In [257]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, R2Plus1D_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,3.147390e-08
1,total-B-line,9.650480e-11
2,Consolidation,1.648905e-02
3,Pleural effusion,4.551105e-05


In [258]:
CIs, AUCs = cal_CI(models_gt, R2Plus1D_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.789274,0.746428,0.832120
1,total-B-line,0.676343,0.616471,0.736215
2,Consolidation,0.830866,0.785641,0.876091
3,Pleural effusion,0.869231,0.819084,0.919378


### LUVM vs CNN Only(batch 32 / batch 128)

In [259]:
# batch 128
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, densenet161_128_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.069983
1,total-B-line,0.084226
2,Consolidation,0.177298
3,Pleural effusion,0.255784


In [260]:
CIs, AUCs = cal_CI(models_gt, densenet161_128_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.934159,0.910429,0.957889
1,total-B-line,0.823352,0.781538,0.865165
2,Consolidation,0.858515,0.818131,0.898898
3,Pleural effusion,0.968911,0.944933,0.992888


### LUVM vs CNNLSTM

In [261]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, CNNLSTM_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.000000e+00
1,total-B-line,0.000000e+00
2,Consolidation,5.815957e-07
3,Pleural effusion,1.034528e-04


In [262]:
CIs, AUCs = cal_CI(models_gt, CNNLSTM_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.395210,0.340573,0.449846
1,total-B-line,0.332021,0.275575,0.388466
2,Consolidation,0.757585,0.702562,0.812607
3,Pleural effusion,0.914031,0.875723,0.952339


# test set

In [307]:
fold_num = 3
batch_size = 4
# LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_batch32_fold_3_densenet_161_output.csv
# densenet161_32_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch32_fold_3_densenet161_output.csv')
densenet161_128_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_batch128_fold_{fold_num}_densenet_161_output.csv')

C3D_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_fold_{fold_num}_best_C3D_output.csv')
R2Plus1D_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_fold_{fold_num}_best_R2Plus1D_output.csv')
CNNLSTM_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_fold_{fold_num}_best_CNNLSTM_output.csv')

USVN_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_fold_{fold_num}_best_USVN_output.csv')

LUVM_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_batch4_fold_{fold_num}_LUVM_v4_____8head_13ksize_output.csv')
models_gt = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_fold_{fold_num}_video_level_gt.csv')

## LUVM vs USVN

In [308]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, USVN_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,2.708008e-08
1,total-B-line,9.529656e-02
2,Consolidation,7.167936e-05
3,Pleural effusion,1.646700e-01


In [309]:
CIs, AUCs = cal_CI(models_gt, USVN_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.804182,0.759343,0.849020
1,total-B-line,0.864876,0.828180,0.901572
2,Consolidation,0.874427,0.833620,0.915234
3,Pleural effusion,0.805826,0.684331,0.927321


In [310]:
CIs, AUCs = cal_CI(models_gt, LUVM_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.895135,0.860203,0.930066
1,total-B-line,0.877737,0.843571,0.911903
2,Consolidation,0.825108,0.772883,0.877333
3,Pleural effusion,0.829327,0.727075,0.931580


## LUVM vs cnn only

In [311]:
# batch 128
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, densenet161_128_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.000000
1,total-B-line,0.077471
2,Consolidation,0.541570
3,Pleural effusion,0.031414


In [312]:
CIs, AUCs = cal_CI(models_gt, densenet161_128_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.768451,0.719148,0.817755
1,total-B-line,0.896046,0.864707,0.927384
2,Consolidation,0.813897,0.759348,0.868446
3,Pleural effusion,0.774551,0.627619,0.921483


## LUVM vs C3D

In [313]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, C3D_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,2.389027e-05
1,total-B-line,5.605992e-09
2,Consolidation,4.442550e-02
3,Pleural effusion,3.320136e-03


In [314]:
CIs, AUCs = cal_CI(models_gt, C3D_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.804829,0.745948,0.863709
1,total-B-line,0.741337,0.688002,0.794673
2,Consolidation,0.762425,0.692100,0.832751
3,Pleural effusion,0.904656,0.845602,0.963709


## LUVM vs R2Plus1D

In [315]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, R2Plus1D_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,2.120015e-11
1,total-B-line,1.576517e-14
2,Consolidation,8.511949e-05
3,Pleural effusion,5.931366e-02


## LUVM vs CNNLSTM

In [316]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, CNNLSTM_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.000000
1,total-B-line,0.000000
2,Consolidation,0.007971
3,Pleural effusion,0.018720


In [317]:
CIs, AUCs = cal_CI(models_gt, CNNLSTM_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.443564,0.364044,0.523083
1,total-B-line,0.519239,0.461647,0.576831
2,Consolidation,0.736933,0.676767,0.797098
3,Pleural effusion,0.753552,0.648007,0.859097


# 1D module 실험

### internel test

In [333]:
fold_num = 4
batch_size = 4
LUVM_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch4_fold_{fold_num}_LUVM_v4_____8head_13ksize_output.csv')
LUVM_wo1d_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch4_fold_{fold_num}_LUVM_v4______8head_13ksize_output.csv')
models_gt = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_fold_{fold_num}_video_level_gt.csv')

In [334]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, LUVM_wo1d_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.731133
1,total-B-line,0.302321
2,Consolidation,0.994602
3,Pleural effusion,0.048345


In [335]:
CIs, AUCs = cal_CI(models_gt, LUVM_wo1d_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.926583,0.895952,0.957214
1,total-B-line,0.840140,0.797569,0.882711
2,Consolidation,0.837577,0.796521,0.878633
3,Pleural effusion,0.951531,0.931752,0.971310


### temporally separated test

In [32]:
fold_num = 3
batch_size = 4

LUVM_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_batch4_fold_{fold_num}_LUVM_v4_____8head_13ksize_output.csv')

LUVM_wo1d_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_batch4_fold_3_LUVM_v4______8head_13ksize_output.csv')
# LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_batch4_fold_3_LUVM_v4______8head_13ksize_output.csv
models_gt = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/densenet_161/5_artifacts_fold_3_video_level_gt.csv')

In [33]:
LUVM_wo1d_outputs

,LUVM_A-line,LUVM_total-B-line,LUVM_Consolidation,LUVM_Pleural effusion
0,0.574013,0.336712,0.007180,0.003330
1,0.641353,0.215970,0.009848,0.003551
2,0.661241,0.269350,0.009012,0.003144
3,0.575281,0.335554,0.009190,0.003315
4,0.608442,0.297772,0.008585,0.003745
...,...,...,...,...
387,0.017041,0.973854,0.000799,0.000243
388,0.011602,0.959078,0.002124,0.000441
389,0.009800,0.956933,0.002581,0.000387
390,0.011431,0.962047,0.001887,0.000412


In [34]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, LUVM_wo1d_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.000025
1,total-B-line,0.825813
2,Consolidation,0.042008
3,Pleural effusion,0.135088


In [35]:
CIs, AUCs = cal_CI(models_gt, LUVM_wo1d_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.848748,0.807987,0.889510
1,total-B-line,0.875611,0.840788,0.910434
2,Consolidation,0.786894,0.724913,0.848875
3,Pleural effusion,0.853186,0.759945,0.946426


# Clip length 실험

In [83]:
frame_num = 70

densenet161_128_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/clip_length_ablation_study/{frame_num}frame_5_artifacts_batch128_fold_3_densenet161_output.csv')

C3D_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/clip_length_ablation_study/{frame_num}frame_5_artifacts_batch4_fold_3_C3D_output.csv')
R2Plus1D_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/clip_length_ablation_study/{frame_num}frame_5_artifacts_batch4_fold_3_R2Plus1D_output.csv')
CNNLSTM_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/clip_length_ablation_study/{frame_num}frame_5_artifacts_batch4_fold_3_CNNLSTM_output.csv')

USVN_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/clip_length_ablation_study/{frame_num}frame_5_artifacts_batch4_fold_3_USVN_output.csv')

LUVM_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/clip_length_ablation_study/{frame_num}frame_5_artifacts_batch4_fold_3_LUVM_v4_____8head_13ksize_output.csv')
models_gt = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/clip_length_ablation_study/{frame_num}frame_5_artifacts_fold_3_video_level_gt.csv')

In [84]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, USVN_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.051617
1,total-B-line,0.764991
2,Consolidation,0.298497
3,Pleural effusion,0.259076


In [85]:
# batch 128
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, densenet161_128_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.180965
1,total-B-line,0.794942
2,Consolidation,0.522237
3,Pleural effusion,0.436388


In [86]:
CIs, AUCs = cal_CI(models_gt, densenet161_128_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.920014,0.878677,0.961350
1,total-B-line,0.817674,0.754474,0.880874
2,Consolidation,0.865118,0.814181,0.916054
3,Pleural effusion,0.958815,0.923216,0.994413


In [87]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, C3D_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,1.361293e-07
1,total-B-line,2.908260e-03
2,Consolidation,1.750371e-02
3,Pleural effusion,5.570284e-03


In [88]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, R2Plus1D_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,4.121887e-10
1,total-B-line,6.008288e-04
2,Consolidation,4.628094e-04
3,Pleural effusion,1.776357e-15


In [89]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_outputs, CNNLSTM_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.000000e+00
1,total-B-line,9.905343e-11
2,Consolidation,1.225197e-04
3,Pleural effusion,2.718957e-04


In [90]:
CIs, AUCs = cal_CI(models_gt, LUVM_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/video_base/pda_detection/code/compare_auc_delong.py:297: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.938650,0.907114,0.970186
1,total-B-line,0.811826,0.743703,0.879949
2,Consolidation,0.849679,0.792594,0.906765
3,Pleural effusion,0.971128,0.952205,0.990050


## num heads 실험

In [82]:
head_num = 96
LUVM_v5_outputs2 = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/internal_test/densenet_161/5_artifacts_batch8_fold_3_LUVM_v5_{head_num}head_output.csv')

In [83]:
video_p_value_dict = cal_pvalue(labels, models_gt, LUVM_v5_outputs2, USVN_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.517711
1,total-B-line,0.456481
2,Consolidation,0.000062
3,Pleural effusion,0.000003


In [84]:
CIs, AUCs = cal_CI(models_gt, LUVM_v5_outputs2)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/frame_base/multilabel_classification_train/compare_auc_delong.py:287: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.924062,0.895405,0.952720
1,total-B-line,0.839835,0.800248,0.879423
2,Consolidation,0.891014,0.859087,0.922941
3,Pleural effusion,0.980613,0.970044,0.991181


## Temporally separated test

In [63]:
# cnn_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/{artifacts_num}_artifacts_fold_3_best_2d_output.csv')
C3D_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/{artifacts_num}_artifacts_fold_3_best_C3D_output.csv')
# R2Plus1D_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/{artifacts_num}_artifacts_fold_3_best_R2Plus1D_output.csv')
CNNLSTM_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/{artifacts_num}_artifacts_fold_3_best_CNNLSTM_output.csv')
USVN_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/{artifacts_num}_artifacts_fold_3_best_USVN_output.csv')
video_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/{artifacts_num}_artifacts_fold_3_best_video_model_output.csv')
models_gt = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_level/temporally_separated_test/{artifacts_num}_artifacts_fold_3_video_level_gt.csv')

In [81]:
print(len(cnn_outputs))
cnn_outputs

474


,3_A-line,3_total-B-line,3_Consolidation,3_Pleural effusion
0,0.728043,0.065068,0.078340,0.121497
1,0.691645,0.055179,0.065315,0.112658
2,0.695741,0.052250,0.066525,0.107402
3,0.568470,0.080939,0.129661,0.123249
4,0.450195,0.086349,0.129661,0.118440
...,...,...,...,...
469,0.062679,0.074008,0.719383,0.801811
470,0.068613,0.068203,0.719613,0.773646
471,0.061135,0.060834,0.755646,0.793824
472,0.062256,0.081769,0.809540,0.734797


In [31]:
print(len(video_outputs))
video_outputs

392


,USVN_A-line,USVN_total-B-line,USVN_Consolidation,USVN_Pleural effusion
0,0.321854,0.668210,0.014915,0.011951
1,0.457113,0.528911,0.016151,0.012893
2,0.445416,0.567248,0.015743,0.010894
3,0.304420,0.696578,0.012424,0.008730
4,0.325486,0.663835,0.014844,0.013215
...,...,...,...,...
387,0.024509,0.904883,0.016529,0.013018
388,0.026318,0.913418,0.019589,0.012833
389,0.025962,0.888194,0.024112,0.012140
390,0.026300,0.896127,0.020823,0.013988


In [32]:
print(len(C3D_outputs))
C3D_outputs

392


,C3D_A-line,C3D_total-B-line,C3D_Consolidation,C3D_Pleural effusion
0,0.311652,0.311434,0.145453,0.017238
1,0.318230,0.311335,0.141883,0.016883
2,0.304856,0.304731,0.152616,0.017459
3,0.308637,0.307617,0.146988,0.017092
4,0.314568,0.313499,0.142130,0.016890
...,...,...,...,...
387,0.402936,0.323728,0.088552,0.015070
388,0.392133,0.327725,0.090767,0.014984
389,0.391612,0.330466,0.090299,0.014882
390,0.396780,0.326004,0.089520,0.015007


In [33]:
print(len(R2Plus1D_outputs))
R2Plus1D_outputs

392


,R2Plus1D_A-line,R2Plus1D_total-B-line,R2Plus1D_Consolidation,R2Plus1D_Pleural effusion
0,0.205503,0.328358,0.275765,0.099140
1,0.205293,0.326937,0.286268,0.095453
2,0.207102,0.325053,0.268120,0.095630
3,0.202631,0.327746,0.280932,0.102641
4,0.203766,0.322867,0.275133,0.097144
...,...,...,...,...
387,0.241655,0.332459,0.179309,0.078686
388,0.237187,0.325057,0.190098,0.078645
389,0.234188,0.333842,0.190336,0.077594
390,0.235950,0.331345,0.196463,0.079531


In [34]:
print(len(models_gt))
models_gt

392


,A-line,total-B-line,Consolidation,Pleural effusion
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
387,0,0,1,0
388,0,0,1,0
389,0,0,1,0
390,0,0,1,0


In [22]:
CIs, AUCs = cal_CI(models_gt, video_outputs)
results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/frame_base/multilabel_classification_train/compare_auc_delong.py:287: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.947516,0.927727,0.967305
1,total-B-line,0.837485,0.795940,0.879030
2,Consolidation,0.861883,0.817994,0.905772
3,Pleural effusion,0.965975,0.949206,0.982745


# video model vs cnn(batch 32)

In [11]:
video_p_value_dict = cal_pvalue(labels, models_gt, UVM_outputs, cnn_32_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.011373
1,total-B-line,0.270462
2,Consolidation,0.702711
3,Pleural effusion,0.774596


In [12]:
CIs, AUCs = cal_CI(models_gt, cnn_32_outputs)

/home/work/LUS/code/frame_base/multilabel_classification_train/compare_auc_delong.py:287: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


In [13]:
results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.934159,0.910429,0.957889
1,total-B-line,0.823352,0.781538,0.865165
2,Consolidation,0.858515,0.818131,0.898898
3,Pleural effusion,0.968911,0.944933,0.992888


# video model vs cnn(batch 128)

In [14]:
video_p_value_dict = cal_pvalue(labels, models_gt, UVM_outputs, cnn_128_outputs)
video_p_value_df = pd.DataFrame(list(video_p_value_dict.items()), columns=['Label', 'p-value'])
video_p_value_df

,Label,p-value
0,A-line,0.011373
1,total-B-line,0.270462
2,Consolidation,0.702711
3,Pleural effusion,0.774596


In [15]:
CIs, AUCs = cal_CI(models_gt, cnn_128_outputs)
results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/frame_base/multilabel_classification_train/compare_auc_delong.py:287: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.934159,0.910429,0.957889
1,total-B-line,0.823352,0.781538,0.865165
2,Consolidation,0.858515,0.818131,0.898898
3,Pleural effusion,0.968911,0.944933,0.992888


# video model vs C3D

In [64]:
video_p_value_dict2 = cal_pvalue(labels, models_gt, UVM_outputs, C3D_outputs)
video_p_value_df2 = pd.DataFrame(list(video_p_value_dict2.items()), columns=['Label', 'p-value'])
video_p_value_df2

,Label,p-value
0,A-line,5.781597e-12
1,total-B-line,5.261477e-01
2,Consolidation,1.189325e-03
3,Pleural effusion,6.954405e-01


In [65]:
CIs, AUCs = cal_CI(models_gt, C3D_outputs)

/home/work/LUS/code/frame_base/multilabel_classification_train/compare_auc_delong.py:287: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


In [66]:
results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.702561,0.648675,0.756448
1,total-B-line,0.768549,0.718127,0.818970
2,Consolidation,0.724490,0.649680,0.799299
3,Pleural effusion,0.881656,0.809854,0.953458


# video model vs R2Plus1D

In [38]:
video_p_value_dict3 = cal_pvalue(labels, models_gt, video_outputs, R2Plus1D_outputs)
video_p_value_df3 = pd.DataFrame(list(video_p_value_dict3.items()), columns=['Label', 'p-value'])
video_p_value_df3

,Label,p-value
0,A-line,0.000000e+00
1,total-B-line,9.235370e-09
2,Consolidation,1.522867e-09
3,Pleural effusion,3.500537e-01


In [39]:
CIs, AUCs = cal_CI(models_gt, R2Plus1D_outputs)

/home/work/LUS/code/frame_base/multilabel_classification_train/compare_auc_delong.py:287: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


In [40]:
results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.692604,0.642174,0.743033
1,total-B-line,0.620855,0.564767,0.676942
2,Consolidation,0.660870,0.586503,0.735238
3,Pleural effusion,0.857743,0.792399,0.923087


# video model vs USVN

In [16]:
video_p_value_dict4 = cal_pvalue(labels, models_gt, UVM_outputs, USVN_outputs)
video_p_value_df4 = pd.DataFrame(list(video_p_value_dict4.items()), columns=['Label', 'p-value'])
video_p_value_df4

,Label,p-value
0,A-line,7.628636e-03
1,total-B-line,1.900973e-09
2,Consolidation,3.416010e-02
3,Pleural effusion,1.894995e-01


In [17]:
CIs, AUCs = cal_CI(models_gt, USVN_outputs)

/home/work/LUS/code/frame_base/multilabel_classification_train/compare_auc_delong.py:287: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


In [18]:
results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.883241,0.844990,0.921492
1,total-B-line,0.734219,0.678299,0.790138
2,Consolidation,0.838783,0.800481,0.877085
3,Pleural effusion,0.957209,0.934248,0.980169


In [19]:
CIs, AUCs = cal_CI(models_gt, UVM_outputs)

results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

/home/work/LUS/code/frame_base/multilabel_classification_train/compare_auc_delong.py:287: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.906950,0.871335,0.942564
1,total-B-line,0.842766,0.802981,0.882550
2,Consolidation,0.851954,0.817021,0.886888
3,Pleural effusion,0.966091,0.950675,0.981508


# video model vs CNNLSTM

In [70]:
video_p_value_dict5 = cal_pvalue(labels, models_gt, video_outputs, CNNLSTM_outputs)
video_p_value_df5 = pd.DataFrame(list(video_p_value_dict5.items()), columns=['Label', 'p-value'])
video_p_value_df5

,Label,p-value
0,A-line,0.000000
1,total-B-line,0.000000
2,Consolidation,0.164894
3,Pleural effusion,0.004931


In [71]:
CIs, AUCs = cal_CI(models_gt, CNNLSTM_outputs)

/home/work/LUS/code/frame_base/multilabel_classification_train/compare_auc_delong.py:287: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


In [72]:
results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.273980,0.218758,0.329201
1,total-B-line,0.439123,0.380281,0.497966
2,Consolidation,0.869308,0.827746,0.910870
3,Pleural effusion,0.812374,0.700590,0.924157


# UNM model + frame loss vs cnn only

In [23]:
UVM_frame_loss_outputs = pd.read_csv(f'/home/work/LUS/Results/model_output_csv/video_model2/internal_test/5_artifacts_batch4_fold_3_video_weight0.2_best_UVM_frame_loss_output.csv')

In [24]:
video_p_value_dict6 = cal_pvalue(labels, models_gt, UVM_frame_loss_outputs, cnn_128_outputs)
video_p_value_df6 = pd.DataFrame(list(video_p_value_dict6.items()), columns=['Label', 'p-value'])
video_p_value_df6

,Label,p-value
0,A-line,0.750374
1,total-B-line,0.191483
2,Consolidation,0.336289
3,Pleural effusion,0.445487


In [27]:
CIs, AUCs = cal_CI(models_gt, UVM_frame_loss_outputs)

/home/work/LUS/code/frame_base/multilabel_classification_train/compare_auc_delong.py:287: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  predictions_sorted_transposed = predictions[np.newaxis, order]


In [28]:
results = {
    'Label': list(AUCs.keys()),
    'AUC': list(AUCs.values()),
    '95% CI Lower Bound': [ci[0] for ci in CIs.values()],
    '95% CI Upper Bound': [ci[1] for ci in CIs.values()]
}
results_df = pd.DataFrame(results)
results_df

,Label,AUC,95% CI Lower Bound,95% CI Upper Bound
0,A-line,0.936260,0.910475,0.962045
1,total-B-line,0.841545,0.800874,0.882215
2,Consolidation,0.842076,0.797593,0.886560
3,Pleural effusion,0.975476,0.962593,0.988359


In [7]:
encoder.classifier

Linear(in_features=2208, out_features=1000, bias=True)

In [8]:
encoder2 = timm.create_model('densenet161', pretrained=False, num_classes=300)
encoder2.classifier

Linear(in_features=2208, out_features=300, bias=True)